In [7]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt

from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.layers import SimpleRNN, Dense, Flatten, LSTM, Reshape, Conv2D, MaxPooling2D, Input, Dropout
from keras.optimizers import Adam

from google.colab import drive
drive.mount('/content/gdrive')

n_output = 784
n_noise = 128

generator = Sequential()
generator.add(Dense(units = 256, input_dim = n_noise, activation = 'relu'))
generator.add(Dense(units = 512, activation = 'relu'))
generator.add(Dense(units = n_output, activation = 'tanh'))

discriminator = Sequential()
discriminator.add(Dense(256, input_dim = n_output, activation = 'relu'))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation = 'sigmoid'))

Mounted at /content/gdrive


In [8]:
g_input = Input(shape = (n_noise, ))
g_output = discriminator(generator(g_input))
gan = Model(g_input, g_output)

discriminator.trainable = True
adam = tf.optimizers.Adam(learning_rate = 0.0002, beta_1 = 0.5)
discriminator.compile(loss = 'binary_crossentropy', optimizer = adam)

discriminator.trainable = False
gan.compile(loss = 'binary_crossentropy', optimizer = adam)

gan.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128)]             0         
                                                                 
 sequential_4 (Sequential)   (None, 784)               566800    
                                                                 
 sequential_5 (Sequential)   (None, 1)                 201217    
                                                                 
Total params: 768,017
Trainable params: 566,800
Non-trainable params: 201,217
_________________________________________________________________


In [ ]:
n_epoch = 100
batch_size = 128
Saving_interval = 10

(train_images, _), (_, _) = mnist.load_data()
n_images = len(train_images)
train_images = train_images.reshape((n_images, n_output))
train_images = (train_images - 127.5) / 127.5

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(n_images).batch(batch_size)

for i in range(n_epoch):
  print("Epoch Num : {}/{}".format((i+1), n_epoch))
  for image_batch in train_dataset:
    n_bt_imgs = len(image_batch)
    true_labels = np.ones((n_bt_imgs, 1))
    fake_labels = np.zeros((n_bt_imgs, 1))
    d_loss_real = discriminator.train_on_batch(image_batch, true_labels)

    noise = np.random.normal(0, 1, (n_bt_imgs, n_noise))
    gen_imgs = generator.predict(noise)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake_labels)

    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    #gan 모델 학습 
    g_loss = gan.train_on_batch(noise, true_labels)
    
  if (i + 1) % Saving_interval == 0 or i == 0:
    print('Epoch : %d' % (i + 1), 'd_loss : %.4f' % d_loss, 'g_loss : %.4f' % g_loss)
    noise = np.random.normal(0, 1, (25, n_noise))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5
    fig, axs = plt.subplots(5, 5)
    count = 0
    for j in range(5):
      for k in range(5):
        axs[j, k].imshow(np.reshape((gen_imgs[count]), (28, 28)))
        axs[j, k].axis('off')
        count += 1
    plt.show()
    fig.savefig("/content/gdrive/My Drive/gan_results/g_mnist_%d.png"%(i+1)) 

In [18]:
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import functools
import os

#load image
def load_image(image_path):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_image(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, (256,256))
  image = image[tf.newaxis,:] 
  return image

def tensor2image(image):
  image = image*255.0
  image = np.array(image, dtype=np.uint8)
  return Image.fromarray(image)

content_image = load_image('/content/gdrive/My Drive/content1.png')
style_image = load_image('/content/gdrive/My Drive/style1.jpg')

hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0][0]
#stylized_image = outputs[0][0]

final_image = tensor2image(stylized_image)
if final_image.mode != 'RGB':
    final_image = final_image.convert('RGB')
final_image.save('/content/gdrive/My Drive/stylized_image1.jpg')

NotFoundError: ignored